# Import Libraries

In [ ]:
import socket
import pickle
import numpy as np
import pandas as pd
import json 
import warnings
import matplotlib.pyplot as plt
from openpyxl import Workbook


warnings.filterwarnings('ignore')

# Send And Receive Func for Huge Data Size

In [ ]:
# Receive data until fully received (For Huge data => chunk by chunk)
def receive_data(socket, length):
    data = b''
    while len(data) < length:
        packet = socket.recv(length - len(data))
        if not packet:
            return None
        data += packet
    return data

# Send data in chunks(For Huge data => chunk by chunk)
def send_data(socket, data):
    data_pickle = pickle.dumps(data)
    data_size = len(data_pickle)
    socket.sendall(data_size.to_bytes(4, 'big'))  # Send data size first

    sent = 0
    while sent < data_size:
        chunk = data_pickle[sent:sent+4096]  # Send in chunks
        socket.sendall(chunk)
        sent += len(chunk)



#  Initialize server socket

In [ ]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ('localhost', 1111)
server_socket.bind(server_address)
server_socket.listen(5)

# Set Federated Learning Server Parameters

In [ ]:
# Maximum number of server_rounds or aggregation local models
server_rounds = 30   

client_num = 10
clients = []
client_params = {}

# Define Local Models Structure

In [ ]:
num_features = 784
n0 = num_features
n1 = 128
n2 = 10

# Initialize Client Paramters 

In [ ]:
# Initialize weights
np.random.seed(100)
a = -1
b = 1
w1 = np.random.uniform(a, b, size=(n1, n0))
w2 = np.random.uniform(a, b, size=(n2, n1)) 

client_epochs = 10

# Optimization method params
optimizer = 'rmsprop'

# Load optimization_params from a JSON file
with open('optimization_params.json', 'r') as file:
    optimization_params = json.load(file) 

optimizer_plot_config = {
        "sgd": {
            "title":"FedSGD baseline(simple minibatch updates )",
            "xlabel":"iterations",
            "ylabel":"Accuracy",
            "colors":['blue', 'cyan', 'green', 'yellow', 'red','gray']
        },
        "nesterov_momentum":{
            "title":"FedSGD + Nesterov momentum",
            "xlabel":"iterations",
            "ylabel":"Accuracy",
            "colors":["#000000","#A14BAD","#6514A6","#0000C5","#0054DD","#0095DD","#00AAA8","#00A668","#00A700","#00CF00","#00F700","#EBFFC0","#F0EA00","#FFBD00","#FF5100","#F16262","#CF0000","#F3F3F3"]
        },
        "adagrad":{
            "title":"FedSGD with AdaGrad",
            "xlabel":"iterations",
            "ylabel":"Accuracy",
            "colors":["#000000","#0000DD","#00AA87","#01FF00","#FF9800","#CCCCCC"]
        },
        "rmsprop":{
            "title":"FedSGD with RMSProp",
            "xlabel":"iterations",
            "ylabel":"Accuracy",
            "colors":["#000000","#79008A","#57009E","#0000C5","#0053DD","#0094DD","#00AAA7","#00AAA7","#00AAA7","#00CF00","#00F700","#AFFF00","#F0EA00","#FFBC00","#FF5100","#E90000","#CF0000","#CF0000"]
        },
        "adam":{
            "title":"Federated Averaging - Adam",
            "xlabel":"iterations",
            "ylabel":"Accuracy",
            "colors":["#000000","#4B2D4F","#804389","#7A048A","#7F0090","#992AA7","#9547B7","#3E08A4","#0200A9","#2727C6","#4A4ADE","#0B1CDE","#0038DD","#247BE1","#4EA6E7","#0E92DE","#0099DB","#20ABCD","#51C1CD","#12B0AB","#00AA97","#1DB397","#54C192","#15A539","#009900","#19AE19","#58CD58","#19C719","#00CF00","#16DF16","#55EF55","#1CF51C","#0FFF00","#63FF13","#BFFF52","#CDFA1F","#D8F500","#EDEF0F","#F7EA4E","#FAD923","#FFC500","#FFB30C","#FFBB4B","#FF8926","#FF3900","#FE0808","#F44848","#E72A2A","#DB0000","#D40000","#DA4444","#D55D5D","#CC8B8B","#CCCCCC"]
        }
    }

initial_parameters = {
    'data_mode':'full-non-iid',#'full-non-iid'|'99-non-iid'
    'optimizer': optimizer,
    'optimization_params': optimization_params[optimizer],
    'weights': [[w1, w2] for _ in range(len(optimization_params[optimizer]))],
    'client_epochs':client_epochs,
    'server_rounds': server_rounds,
    'batch_size':24,
    'lambda_reg':0.00001
}

# Initialize lists to store aggregated test accuracies for each optimizer
aggregated_test_accuracies = np.zeros((server_rounds, len(optimization_params[optimizer])))


# Federated Learning Process Handling On Server

In [ ]:
print('**************************************************************************************************')
print('*********************************** WATTING FOR CONNECTIONS **************************************')
print('**************************************************************************************************')

while len(clients) < client_num: 
    client, client_address = server_socket.accept()
    clients.append(client)
    print(f"Connection from {client_address}")
    
    # Send initial params for current iteration to the clients
    send_data(client, initial_parameters)
    
print("Sent Initial Params To All Clients")

print("\n\033[1;m" + "*" * 125)

current_round = 0 
while current_round < server_rounds:
    # Send ready signal to clients
    for client in clients:
        client.send(pickle.dumps({'ready': True}))
    
    print("Start Round",current_round+1)
    ############################################################
    #  Wait For Local Model Train Results
    ############################################################
    # Receive updated train params from clients for the next round
    client_train_res = {}
    print('Pending Receive Params From Clients For Next Round')
    
    for client in clients:
        data_size = int.from_bytes(client.recv(4), 'big')  # Receive data size first
        data = receive_data(client, data_size) 
        client_train_res[client.getpeername()] = pickle.loads(data)
        print('Received From ClientID: ',client_train_res[client.getpeername()]["client_id"])
        print('Train MSE: ',client_train_res[client.getpeername()]["train_mse"],'  ***  ', 'Train Accuracy: ',client_train_res[client.getpeername()]["train_accuracy"]) 

    
    ############################################################
    #  Aggregate weights based on the number of samples
    ############################################################
    
    print("Start Aggregation Process")
    total_samples = sum(client_train_res[addr]['num_samples'] for addr in client_train_res)
    # Calculate weighted average of params from clients
    aggregated_parameters = {
        'weights': [[np.zeros_like(w1), np.zeros_like(w2)] for _ in range(len(optimization_params[optimizer]))],
        'current_server_round':current_round
        }
    
    for addr, client_train_data in client_train_res.items():
        ratio = client_train_data['num_samples'] / total_samples 
        for i in range(len(optimization_params[optimizer])): 
            for j in range(2):
                aggregated_parameters['weights'][i][j] += client_train_data['weights'][i][j] * ratio  
    
    print("End Aggregation Process")
    
    
    
    ############################################################
    #  Send updated model to clients for evaluate by test data
    ############################################################
    for client in clients:
        send_data(client, aggregated_parameters)
    print("Sent Aggregated Model Params To Client To Evaluate On Test Data")   
    
    print('Pending Receive Test results From Clients')
    # Receive evaluate global model on test-result from clients 
    client_test_res = {}
    for client in clients: 
        data_size = int.from_bytes(client.recv(4), 'big')  # Receive data size first
        data = receive_data(client, data_size)
        client_test_res[client.getpeername()] = pickle.loads(data)
        print('Received From ClientID: ',client_test_res[client.getpeername()]["client_id"])
        print('Test MSE: ',client_test_res[client.getpeername()]["test_mse"],'  ***  ', 'Test Accuracy: ',client_test_res[client.getpeername()]["test_accuracy"]) 

    
    # Aggregate test accuracies for each optimizer
    for i in range(len(optimization_params[optimizer])):
        aggregated_test_accuracies[current_round, i] = np.average(
            [client_test_res[addr]['test_accuracy'][i] for addr in client_test_res],
            weights=[client_train_data['num_samples'] for client_train_data in client_train_res.values()]
        )
    
    print('aggregated_test_accuracies: ', aggregated_test_accuracies)
    
    
    # Plot test accuracies for each optimizer conditions
    plt.figure(figsize=(10, 6))
    for i in range(len(optimization_params[optimizer])):
        plt.plot(range(1, current_round + 2), aggregated_test_accuracies[:current_round + 1, i], label=optimization_params[optimizer][i]["description"], color=optimizer_plot_config[optimizer]["colors"][i])

    plt.title(optimizer_plot_config[optimizer]["title"])
    plt.xlabel('iterations')
    plt.ylabel('Accuracy')
    # Display legend outside the plot
    plt.legend(loc='upper right', bbox_to_anchor=(1.41, 1), prop={'size': 5})
    plt.show()
    
    print("End Round",current_round+1)
    print("\n\033[1;m" + "*" * 125)
    
    current_round += 1
    
# Send termination signal to clients
for client in clients:
    client.send(pickle.dumps({'terminate': True}))

# Close connections
server_socket.close()

print('**************************************************************************************************')
print('******************************************* FL END ***********************************************')
print('**************************************************************************************************')

# Save Accuracies in Excel

In [ ]:
data = []
for i, row in enumerate(aggregated_test_accuracies):
    for j, acc in enumerate(row): 
        description = optimization_params[optimizer][j]["description"] 
        data.append([i+1, description, acc])

df = pd.DataFrame(data, columns=["Round", "Description", "Accuracy"])

# Create a new workbook
wb = Workbook()

# Select active worksheet
ws = wb.active

# Add headers
headers = ["Round", "Description", "Accuracy"]
ws.append(headers)

# Add data
for row in data:
    ws.append(row)

# Save the workbook
file_name = optimizer + "_test_accuracies_" + initial_parameters["data_mode"] +".xlsx"
wb.save(file_name) 